# GridPath Workshop (UCSB)

Initialization

In [ ]:
import os
import sys
import pandas as pd

# Print the current working directory. The result should end with state_model
software_dir = os.getcwd()
os.chdir("..")
print(software_dir)

In [ ]:
#Setting this as the primary directory for all future use
# This is be the main folder where all files are created and stored in. 
directory = os.getcwd()
print(directory)

In [ ]:
# Defining the database directory - where the database file will be created and stored. 
db_dir = f"{directory}/db"

# Defining CSV inputs directory - where the state level input files are stored. 
csv_dir = f"{directory}/gridpath_input_csvs"

#Defining scenario folder location
scen_dir = f"{directory}/scenarios"

# GridPath: Production Cost
The following section is for running a production cost model in GridPath.

## User edit section
Add the folder name with input csvs and provide a name for this database.

In [ ]:
# Speciific  model for this run.
# Add the name of the specific mode folder file by replacing the worth between " " 
gridpath_input_csvs = "2028_12-prodcost"

# Add your database name below between "" or leave blank to use input_csvs_gridpath as database name
db_name = ""
db_name = db_name if db_name else gridpath_input_csvs
print(f"Database name is set to: {db_name}")

### Creating database, importing state inputs and importing scenarios to the database for modeling

In [ ]:
!python {directory}/gridpath/db/create_database.py --database {db_dir}/{db_name}.db

In [ ]:
!python {directory}/gridpath/db/utilities/port_csvs_to_db.py --database {db_dir}/{db_name}.db --csv_location {csv_dir}/{gridpath_input_csvs}

In [ ]:
!python {directory}/gridpath/db/utilities/scenario.py --database {db_dir}/{db_name}.db --csv_path {csv_dir}/{gridpath_input_csvs}/scenarios.csv

In [ ]:
# This will print out all possible scenarios available
scenario_csv = pd.read_csv(f"{csv_dir}/{gridpath_input_csvs}/scenarios.csv")
column_names = scenario_csv.columns[1:]

# Print scenario names
print("Available scenarios:")
for name in column_names:
    print(name)

## Select scenarios from the list above and add to the `selected_scenario` below before running.

### User Edit Section

In [ ]:
# Add or remove scenarios from above to the list below:
scen_list = ["2028_12-prodcost-MP"]

### Running selected scenarios

In [ ]:
# for selected_scenario in scen_list:
#     !python {directory}/gridpath/gridpath/run_end_to_end.py --database {db_dir}/{db_name}.db --scenario {selected_scenario} --solver cbc --solver_executable {directory}\solver\CBC\bin\cbc.exe --scenario_location {scen_dir}

# For Mac user, run below:
for selected_scenario in scen_list:
   !python {directory}/gridpath/gridpath/run_end_to_end.py --database {db_dir}/{db_name}.db --scenario {selected_scenario} --scenario_location {scen_dir}

# Visualization: Production Cost
The following section is for visualizing the results from the GridPath Production Cost model above.

In [ ]:
# Checking the directory is correct - should be state_model folder
os.chdir(software_dir)
print(os.getcwd())

In [ ]:
# loading packages and setting results folder
from loading_utils import * 
from plotting_utils import * 
from processing_utils import * 
from pdf_report import * 

pd.options.display.float_format = "{:,.2f}".format

path_to_scenarios = f"{directory}/scenarios"
print(path_to_scenarios)

Loading necessary functions for the zones, scenarios labels `<scenario_labels.csv>` and technology colors `<technology_labels.csv>` to plot the graph for new and existing capacity

In [ ]:
scen_labels_     = pd.read_csv(directory + '/gridpath_input_csvs/scenario_labels-prodcost.csv') 
tech_labels_     = pd.read_csv(directory + '/gridpath_input_csvs/technology_labels-prodcost.csv') 
dispatch_labels_ = pd.read_csv(directory + '/gridpath_input_csvs/dispatch_labels-prodcost.csv') 

Load storage devices state of charge by tehcnology

In [ ]:
soc_ = _load_stor(scen_labels_, path_to_scenarios, gp_model = 'production')

In [ ]:
_plot_soc(soc_, scen_labels_, tech_labels_,
          units_label  = 'State of Charge (%)',
          legend       = True,
          save         = True,
          title        = '',
          file_name    = directory + r'/plots/state_of_charge_prodcost.png')

Load system load, overgenarion and unserved energy

In [ ]:
demand_ = _load_resource_adequacy_demand(scen_labels_, path_to_scenarios, gp_model = 'production')

## Probabilistic reliability metrics for resource adequacy:

* **Loss-Of-Load Probability (LOLP)**: the likelihood that electricity demand will exceed available generation capacity over a specific period.
* **Loss-Of-Load Event (LOLE)**: the expected hours when electricity demand exceeds what is available.
* **Loss-Of-Load Frequency (LOLF)**: the expected number of times a power system experiences a loss-of-load event over the period
* **Loss-Of-Load Duration (LOLD)**: the average length of time that a loss-of-load event lasts when it occurs over the period.
* **Expected Unserved Energy (EUE)**: the amount of demand that is expected to be unmet due to insufficient generation capacity, transmission constraints, or other system limitations over the period.

In [ ]:
summary_ = _resource_adequacy(demand_)
summary_

In [ ]:
capacity_         = _load_capacity(scen_labels_, path_to_scenarios, gp_model = 'production')
grouped_capacity_ = _group_capacity_technologies(capacity_, tech_labels_)
grouped_capacity_

Plotting the new and existing capacity across different scenarios. Add title name for the plot.

In [ ]:
_plot_new_and_existing_capacity(grouped_capacity_, scen_labels_, tech_labels_,
                                units        = 1e3,
                                units_label  = 'Existing & New Capacity (GW)',
                                legend       = True,
                                save         = True,
                                y_grid_inc   = 25,
                                title        = '',
                                file_name    = directory + r'/plots/group_installed_capacity_prodcost.png')

Energy dispatch by scenarios, zone and date

In [ ]:
ed_ = _processing_energy_dispatch(scen_labels_, path_to_scenarios, model = 'production')
ed_ = _group_dispatch_technologies_by_zone_and_date_production(ed_, tech_labels_)

In [ ]:
# Plot energy dispatch for a given day
_plot_zone_energy_dispatch_production(ed_, scen_labels_, tech_labels_, dispatch_labels_,
                                      save      = True,
                                      legend    = True,
                                      file_name = directory + r'/plots/energy_dispatch_prodcost.png')

Load energy dispatch across scenarios and group the energy dispatch by technology

In [ ]:
dispatch_         = _load_energy_dispatch(scen_labels_, path_to_scenarios, gp_model = 'production')
grouped_dispatch_ = _group_dispatch_technologies(dispatch_, tech_labels_)
grouped_dispatch_

Plotting energy dispatch per technology for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_dispatch(grouped_dispatch_, scen_labels_,  tech_labels_, 
               units        = 1e6, 
               units_label  = r'Electricity Generation (TWh)',
               save         = True,
               legend       = True,
               y_grid_inc   = 50,
               title        = '',
               file_name    = directory + r'/plots/group_electricity_generation_prodcost.png')

Load Greenhouse Gas (GHG) emissions and load energy demand for different scenarios

In [ ]:
emissions_, demand_ = _load_GHG_emissions(scen_labels_, path_to_scenarios, gp_model = 'production')
emissions_

Plot GHG emissions for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_emissions(emissions_, scen_labels_, 
                units      = 1e6,
                unit_label = r'GHG Emissions (MtCO$_2$)',
                save       = True,
                legend     = True,
                title      = '',
                file_name  = directory + r'/plots/GHG_emissions_prodcost.png')

Processing GHG emissions intensity for different scenarios

In [ ]:
intesity_, demand_ = _GHG_emissions_intensity(emissions_, demand_)
intesity_

Plot GHG emissions intensity for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_emissions_intensity(intesity_, scen_labels_, 
                          unit_label = r'GHG Intensity (MtCO$_2$/MWh)',
                          save       = True,
                          legend     = True,
                          title      = '',
                          file_name  = directory + r'/plots/GHG_emissions_intensity_prodcost.png')

Load system cost data for different scenarios

In [ ]:
system_cost_ = _load_system_cost(scen_labels_, path_to_scenarios, gp_model = 'production')
system_cost_

Plot system Levelized Cost of Electricity (LCOE) for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_system_cost(system_cost_, scen_labels_, 
                  unit_label = r'Costs (Rupees per MWh)', 
                  save       = True, 
                  legend     = True,
                  title      = '', 
                  file_name  = directory + r'/plots/system_cost_prodcost.png')

In [ ]:
# Replace with a specific scenario if needed, otherwise leave blank
scenario = '2028_12-prodcost-MP' 
# Replace with 'new' or existing' so only new capacity is shown, otherwise leave blank for new and existing
status = '' 
filtered_capacity_ = _filter_capacity(grouped_capacity_, scenario, status)
filtered_capacity_

## PDF Report Output

In [ ]:
# Option to add tables at the end of the PDF
add_tables = True  # Set to True to add tables; False to exclude tables

In [ ]:
# Define file paths based on the existing directory variable
plot_details_csv = directory + r'/gridpath_input_csvs/plot_details-prodcost.csv'
output_pdf_filename = directory + r'/Gridpath_Production_Cost_Report.pdf' 

tables = [summary_]  # List of DataFrames

# Generate the PDF
generate_pdf_report(directory, plot_details_csv, output_pdf_filename, tables=tables, add_tables=add_tables)